In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
def generate_sequences(n_sequences, seq_length):
    x_data = []
    y_data = []

    for _ in range(n_sequences):
        x = np.random.randint(0, 10, seq_length)
        y = np.zeros(seq_length, dtype=int)
        y[0] = x[0]
        for i in range(1, seq_length):
            y[i] = x[i] + x[0]
            if y[i] >= 10:
                y[i] -= 10
        x_data.append(x)
        y_data.append(y)

    return np.array(x_data), np.array(y_data)

n_sequences = 10000
seq_length = 10
x_data, y_data = generate_sequences(n_sequences, seq_length)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)


In [ ]:
# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out)
        return out


In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            inputs, targets = inputs.unsqueeze(-1), targets.unsqueeze(-1)  # Add feature dimension

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Hyperparameters
input_size = 1
hidden_size = 128
output_size = 1
num_layers = 1
num_epochs = 20
learning_rate = 0.001

# Initialize models, criterion and optimizer
rnn_model = RNNModel(input_size, hidden_size, output_size, num_layers)
lstm_model = LSTMModel(input_size, hidden_size, output_size, num_layers)
gru_model = GRUModel(input_size, hidden_size, output_size, num_layers)

criterion = nn.MSELoss()
rnn_optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
gru_optimizer = optim.Adam(gru_model.parameters(), lr=learning_rate)

# Train RNN model
print("Training RNN model...")
train_model(rnn_model, train_loader, criterion, rnn_optimizer, num_epochs)

# Train LSTM model
print("Training LSTM model...")
train_model(lstm_model, train_loader, criterion, lstm_optimizer, num_epochs)

# Train GRU model
print("Training GRU model...")
train_model(gru_model, train_loader, criterion, gru_optimizer, num_epochs)


Training RNN model...
Epoch 1/20, Loss: 7.4475
Epoch 2/20, Loss: 8.0929
Epoch 3/20, Loss: 7.6797
Epoch 4/20, Loss: 7.2571
Epoch 5/20, Loss: 6.1475
Epoch 6/20, Loss: 7.4330
Epoch 7/20, Loss: 6.4687
Epoch 8/20, Loss: 6.0972
Epoch 9/20, Loss: 4.8587
Epoch 10/20, Loss: 3.8956
Epoch 11/20, Loss: 2.8299
Epoch 12/20, Loss: 2.6144
Epoch 13/20, Loss: 1.7338
Epoch 14/20, Loss: 1.5760
Epoch 15/20, Loss: 1.3258
Epoch 16/20, Loss: 1.0955
Epoch 17/20, Loss: 0.9770
Epoch 18/20, Loss: 0.7092
Epoch 19/20, Loss: 0.7121
Epoch 20/20, Loss: 0.7850
Training LSTM model...
Epoch 1/20, Loss: 7.9192
Epoch 2/20, Loss: 7.4242
Epoch 3/20, Loss: 6.7655
Epoch 4/20, Loss: 6.5239
Epoch 5/20, Loss: 4.5657
Epoch 6/20, Loss: 4.3127
Epoch 7/20, Loss: 3.2796
Epoch 8/20, Loss: 1.9794
Epoch 9/20, Loss: 1.6862
Epoch 10/20, Loss: 1.4411
Epoch 11/20, Loss: 1.1532
Epoch 12/20, Loss: 0.9149
Epoch 13/20, Loss: 0.7703
Epoch 14/20, Loss: 0.5421
Epoch 15/20, Loss: 0.7799
Epoch 16/20, Loss: 0.3479
Epoch 17/20, Loss: 0.3210
Epoch 18/20

In [ ]:
def evaluate_and_print_predictions(model, test_loader, num_examples=5):
    model.eval()
    with torch.no_grad():
        inputs, targets = next(iter(test_loader))
        inputs, targets = inputs.unsqueeze(-1), targets.unsqueeze(-1)  # Добавляем измерение признаков

        outputs = model(inputs)

        for i in range(min(num_examples, inputs.size(0))):
            input_seq = inputs[i].squeeze().numpy().astype(int)
            target_seq = targets[i].squeeze().numpy().astype(int)
            predicted_seq = torch.round(outputs[i].squeeze()).numpy().astype(int)

            print(f"Input sequence  : {input_seq}")
            print(f"Target sequence : {target_seq}")
            print(f"Predicted sequence : {predicted_seq}\n")

# Вывод предсказаний для RNN
print("RNN predictions:")
evaluate_and_print_predictions(rnn_model, test_loader)

# Вывод предсказаний для LSTM
print("LSTM predictions:")
evaluate_and_print_predictions(lstm_model, test_loader)

# Вывод предсказаний для GRU
print("GRU predictions:")
evaluate_and_print_predictions(gru_model, test_loader)

RNN predictions:
Input sequence  : [1 8 4 8 7 7 4 2 8 9]
Target sequence : [1 9 5 9 8 8 5 3 9 0]
Predicted sequence : [0 5 5 7 9 9 5 3 8 1]

Input sequence  : [1 5 3 9 6 9 2 1 3 0]
Target sequence : [1 6 4 0 7 0 3 2 4 1]
Predicted sequence : [0 6 4 1 7 1 3 2 4 1]

Input sequence  : [5 9 7 3 8 6 6 2 5 2]
Target sequence : [5 4 2 8 3 1 1 7 0 7]
Predicted sequence : [6 3 2 9 3 1 1 7 1 8]

Input sequence  : [1 1 8 2 2 5 5 7 6 3]
Target sequence : [1 2 9 3 3 6 6 8 7 4]
Predicted sequence : [0 2 8 3 3 6 6 9 7 4]

Input sequence  : [1 6 6 4 3 9 5 4 6 2]
Target sequence : [1 7 7 5 4 0 6 5 7 3]
Predicted sequence : [0 8 7 5 4 1 6 5 7 3]

LSTM predictions:
Input sequence  : [1 8 4 8 7 7 4 2 8 9]
Target sequence : [1 9 5 9 8 8 5 3 9 0]
Predicted sequence : [1 5 5 9 8 8 5 3 9 0]

Input sequence  : [1 5 3 9 6 9 2 1 3 0]
Target sequence : [1 6 4 0 7 0 3 2 4 1]
Predicted sequence : [1 6 4 0 7 0 3 2 4 1]

Input sequence  : [5 9 7 3 8 6 6 2 5 2]
Target sequence : [5 4 2 8 3 1 1 7 0 7]
Predicted sequenc

In [ ]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [ ]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2024-07-20 18:46:07--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.234.240, 52.217.232.24, 52.216.56.40, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.234.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt.2’

nietzsche.txt.2     100%[===================>] 586.82K  1.83MB/s    in 0.3s    

2024-07-20 18:46:08 (1.83 MB/s) - ‘nietzsche.txt.2’ saved [600901/600901]



In [ ]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [ ]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [ ]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [ ]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [ ]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]),
 tensor(23))

In [ ]:
BATCH_SIZE = 512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnn_type, input_dim, hidden_dim, output_dim, num_layers=1):
        super(NeuralNetwork, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = rnn_type(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).cuda()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).cuda()
        out, _ = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, len(CHAR_TO_INDEX)).cuda()

In [ ]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN, len(CHAR_TO_INDEX)), dtype=torch.float32)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t, CHAR_TO_INDEX[char]] = 1.0

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [ ]:
generate_text()

itself in distress the germans invented |ac eqxltsqugeeroyuptedlyzeydtixtyorqo hr


In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        # Преобразование X_b в 3D-тензор
        X_b_one_hot = torch.zeros(X_b.size(0), X_b.size(1), len(CHAR_TO_INDEX)).cuda()
        X_b_one_hot.scatter_(2, X_b.unsqueeze(2), 1)
        optimizer.zero_grad()
        answers = model(X_b_one_hot)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

<ipython-input-90-34ca7b46d8b8>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for X_b, y_b in tqdm.tqdm_notebook(data):


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 0. Time: 3.147, Train loss: 2.676
 whom laboriousness from generation to g|itnes ouerleultode oio topetalne it ne o


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 1. Time: 3.012, Train loss: 2.235
issecting hand or as resolute will to da|limase bemesny nes tose baces me lnterci


...

Epoch 97. Time: 3.242, Train loss: 1.306
t he pins his faith to it must however b|e corripshtions of extraysical relulain 


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 98. Time: 3.840, Train loss: 1.306
emblance of which it is the symptom mora|l what wish is conjuct and hepposion the


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 99. Time: 3.164, Train loss: 1.305
all nature there is no perfectly straigh|e as wect an intramptes that when that o
